In [1]:
import org.apache.spark.rdd.RDD
import org.apache.spark.SparkContext
import geotrellis.shapefile.ShapeFileReader
import geotrellis.spark._
import geotrellis.raster._
import org.apache.spark.sql.{Dataset, SparkSession}
import geotrellis.vector._

In [2]:
import edu.upf.inequality.pipeline.GroupByShape._
import edu.upf.inequality.pipeline.IO._
import edu.upf.inequality.pipeline.Growth._
import edu.upf.inequality.pipeline.Wealth._

In [8]:
implicit val isc = sc
val Array(tilePath, shapeKey, shapeId, maxTileSize, nlKeyA, nlKeyB, popKey, crush, topCode, outFile) = Array("upf-inequality-raw-geotifs", "municipalities/2006-int.tif", "false", "256", "2012-float.tif", "2013-float.tif", "pop-2013-float.tif", "4", "99999999", "growth-out-2")

In [4]:
val tileSize = maxTileSize.toInt
val pop = readRDD(tilePath, popKey, tileSize)

Name: java.lang.InterruptedException
Message: null
StackTrace:   at java.util.concurrent.locks.AbstractQueuedSynchronizer.doAcquireSharedInterruptibly(AbstractQueuedSynchronizer.java:998)
  at java.util.concurrent.locks.AbstractQueuedSynchronizer.acquireSharedInterruptibly(AbstractQueuedSynchronizer.java:1304)
  at scala.concurrent.impl.Promise$DefaultPromise.tryAwait(Promise.scala:202)
  at scala.concurrent.impl.Promise$DefaultPromise.ready(Promise.scala:218)
  at scala.concurrent.impl.Promise$DefaultPromise.ready(Promise.scala:153)
  at org.apache.spark.scheduler.DAGScheduler.runJob(DAGScheduler.scala:619)
  at org.apache.spark.SparkContext.runJob(SparkContext.scala:1918)
  at org.apache.spark.SparkContext.runJob(SparkContext.scala:1981)
  at org.apache.spark.rdd.RDD$$anonfun$reduce$1.apply(RDD.scala:1025)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
  at org.apa

In [5]:
val Seq(wA, wB) = Seq(nlKeyA, nlKeyB).map(readRDD(tilePath, _, tileSize)).map(wealthRaster(_, pop, crush.toFloat, topCode.toFloat))

Name: Unknown Error
Message: lastException: Throwable = null
<console>:55: error: not found: value pop
       val Seq(wA, wB) = Seq(nlKeyA, nlKeyB).map(readRDD(tilePath, _, tileSize)).map(wealthRaster(_, pop, crush.toFloat, topCode.toFloat))
                                                                                                     ^
StackTrace: 

In [6]:
val shapes = if (shapeId != "false") readShapeFile(tilePath, shapeKey, shapeId, wA.metadata) else readRDD(tilePath, shapeKey, tileSize)

Name: Unknown Error
Message: <console>:53: error: not found: value wA
       val shapes = if (shapeId != "false") readShapeFile(tilePath, shapeKey, shapeId, wA.metadata) else readRDD(tilePath, shapeKey, tileSize)
                                                                                       ^
StackTrace: 

In [7]:
implicit val ispark = spark
growth(wA, wB, shapes).collect()

Name: Unknown Error
Message: <console>:51: error: not found: value wA
       growth(wA, wB, shapes).collect()
              ^
<console>:51: error: not found: value wB
       growth(wA, wB, shapes).collect()
                  ^
<console>:51: error: not found: value shapes
       growth(wA, wB, shapes).collect()
                      ^
StackTrace: 

In [182]:
implicit val ispark = spark
val Seq(a,b) = Seq(wA, wB).map(groupByRasterShapz(shapes, _))

val l = a.zip(b).map{ case ((k, v1), (_, v2)) => (k, (v1(0), v2(0)))}
val keys = a.keys.distinct.collect.toList 